# Healthcare System Model

This notbook is then front-end to a model, implemented using Dask/Pandas, of a simple healthcare system to assess the impact of moving activity from acute hospitals into the community. The model is uses historic activity from the acute hospitals and community services as baseline activity within the model. This is then forecast forward using assumptions for population growth by five-year age bands.

## Imported data



In [7]:
import plotly.express as px
from A_data_import import population_data

In [8]:
population_data().data.compute().head().round(3)

age_band   2021   2022   2023   2024   2025   2026   2027   2028   2029  \
0  A.000-004 -0.003 -0.008  0.000 -0.001 -0.001 -0.010 -0.008 -0.006 -0.005   
1  B.005-009  0.002  0.000  0.002  0.011  0.011 -0.025 -0.019 -0.012 -0.010   
2  C.010-014  0.024  0.018  0.005  0.017  0.017 -0.009 -0.023 -0.019 -0.021   
3  D.015-019  0.010  0.032  0.036 -0.001 -0.001  0.024  0.021  0.001  0.002   
4  E.020-024 -0.019 -0.024 -0.010 -0.009 -0.009  0.017  0.030  0.033  0.023   

    2030   2031  
0 -0.004 -0.003  
1 -0.010 -0.009  
2 -0.019 -0.023  
3 -0.004 -0.008  
4  0.023  0.021

In [3]:
from B_forecast_activity import acute_forecast, community_forecast

In [4]:
acute_forecast().data.compute().head()

domain hospital_name region service_line     Sex   age_band  year  activity
0  acute  Hospital one  North   Cardiology  Female  A.000-004  2020   43807.0
1  acute  Hospital one  North   Cardiology  Female  B.005-009  2020   26608.0
2  acute  Hospital one  North   Cardiology  Female  C.010-014  2020   33975.0
3  acute  Hospital one  North   Cardiology  Female  D.015-019  2020   33956.0
4  acute  Hospital one  North   Cardiology  Female  E.020-024  2020   32340.0

In [5]:
cf = community_forecast().data.compute()
cf.head()

domain hospital_name region service_line     Sex   age_band  year  \
0  community  Hospital one  North   Cardiology  Female  A.000-004  2020   
1  community  Hospital one  North   Cardiology  Female  B.005-009  2020   
2  community  Hospital one  North   Cardiology  Female  C.010-014  2020   
3  community  Hospital one  North   Cardiology  Female  D.015-019  2020   
4  community  Hospital one  North   Cardiology  Female  E.020-024  2020   

   activity  
0  219034.0  
1  133041.0  
2  169877.0  
3  169779.0  
4  161700.0

In [7]:
px.bar(
    cf.groupby(["year", "hospital_name"], as_index=False).agg({"activity":"sum"}),
    x="year",
    y="activity",
    color="hospital_name",
    height=720,
    width=1280,
)

In [8]:
px.bar(
    cf.groupby(["year", "service_line"], as_index=False).agg({"activity":"sum"}),
    x="year",
    y="activity",
    color="service_line",
    height=720,
    width=1280,
)

In [9]:
from C_business_transform import business_transform

In [10]:
bt = business_transform().data.compute()
bt.head().round(0)

hospital_name region service_line     Sex   age_band  year  acute_activity  \
0  Hospital one  North   Cardiology  Female  A.000-004  2020         43807.0   
1  Hospital one  North   Cardiology  Female  B.005-009  2020         26608.0   
2  Hospital one  North   Cardiology  Female  C.010-014  2020         33975.0   
3  Hospital one  North   Cardiology  Female  D.015-019  2020         33956.0   
4  Hospital one  North   Cardiology  Female  E.020-024  2020         32340.0   

   community_activity  activity_shift_to_community  revised_acute_activity  \
0            219034.0                          0.0                 43807.0   
1            133041.0                          0.0                 26608.0   
2            169877.0                          0.0                 33975.0   
3            169779.0                          0.0                 33956.0   
4            161700.0                          0.0                 32340.0   

   revised_community_activity  
0                    219034.0  
1                    133041.0  
2                    169877.0  
3                    169779.0  
4                    161700.0

In [11]:
px.line(
    bt.groupby(["year"], as_index=False).sum(),
    x="year",
    y=["acute_activity", "community_activity", "revised_acute_activity", "revised_community_activity"],
    height=720,
    width=1280,
    labels={"value":"Total Annual Activity", "year": "Year"},
)